In [1]:
'''수입원장에서 수금수수료 명세를 추출합니다 '''
import pandas as pd
import GladUtil as gd

ym  = '202504'
# dir0  = f'E:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
dir0 = f'e:/글로벌백업(180329)/04.수수료작업/{ym}수수료/'
         
file0 = f'{ym}_수입원장.xlsx'  #원본

ff = dir0+file0
df = pd.read_excel(ff, sheet_name='피스토스수금', header=3, usecols='a:l').iloc[:13]

### 가온 수금명세 추출

In [2]:
brh = '글래드가온'  #지점
ins = '장기'        #보종
fc_code = ["GLD803173002", "GLD033143019", "GLD803141001", "GLD803143004",
           "GLD033143015", "GLD033143010", "GLD803153002"]
col = ['보험사','증권번호','계약종류','지점','사원번호','사원명','FC코드','계약일자',
       '계약자명','납입회차','상태','보험료','영수일자']
df_all = pd.DataFrame()

print("*한화,흥국만 제목이 '장기고객관리', 기타회사는 모두 '수금'이어야 맞음")
for idx, row in df[['보험사','수금 값열','수금값 제목']].iterrows():
    cols = col + [ row['수금값 제목'] ]
    if row['보험사'] !='계':
        df_temp = pd.DataFrame()
        print(row['보험사'], ':', row['수금값 제목'], " : processing") 
        df_temp = (pd.read_excel(ff,sheet_name=row['보험사'], usecols=cols, dtype={'증권번호': 'object','계약일자': 'object'})
                  .rename(columns={row['수금값 제목']:'총괄수금수수료'}))
        if brh:
            df_temp = df_temp[(df_temp['지점']==brh) & (df_temp['계약종류']==ins)]
        df_all = pd.concat([df_all, df_temp])
        
df_all['지사수금수수료']= df_all['총괄수금수수료'] * .95

df_all['피스토스'] = False
cond = (df_all['사원번호'].isin( fc_code)) & (df_all['계약일자']<'20181101')
df_all.loc[cond,'피스토스'] = True #피스토스 모집건
cond = df_all['피스토스']== True
df_pistos = df_all[df_all['피스토스']== True] # 피스토스 모집건 명세 (정산용df로 복사)

#요약
df_all.loc[cond, '지사수금수수료'] = 0 #피스토스 모집건은 수금 미지급
df_sum = pd.pivot_table(df_all,index=["보험사"],
               values=["지사수금수수료"],
               aggfunc=sum,fill_value=0,margins=True, margins_name="Total").reset_index()

file0 = f'{ym}_글래드가온 수금수수료명세.xlsx'
with pd.ExcelWriter(dir0+file0,mode="w",engine='openpyxl') as writer:  
    df_sum.to_excel(writer, sheet_name='요약',index=False)
    df_all.to_excel(writer, sheet_name='수금명세',index=False)
print("completed... 수금합계= ", df_all['지사수금수수료'].sum())

*한화,흥국만 제목이 '장기고객관리', 기타회사는 모두 '수금'이어야 맞음
AIG손보 : 수금  : processing
DB손보 : 수금관리  : processing
MG손보 : 수금  : processing
흥국화재 : 장기고객관리  : processing
현대해상 : 수금  : processing
한화손보 : 장기고객관리  : processing
KB손보 : 수금  : processing
롯데손보 : 수금  : processing
메리츠화재 : 장기수금  : processing
농협손보 : 수금  : processing
삼성화재 : 장기수금  : processing
하나손보 : 수금수수료  : processing
completed... 수금합계=  2041820.75


#### 피스토스 수금정산표

In [3]:
## 다른방법.. openpyxl 사용
피스토스_수금 = df.iloc[12,11] * 0.95  #수금전체
피스토스_수금2 = df.iloc[12,9] * 0.95  #수금(공동정산대상)
print('피스토스_수금 =', '{:,.0f}'.format(피스토스_수금))
print('피스토스_수금2=', '{:,.0f}'.format(피스토스_수금2))

#피스->가온 이동사원의 수금(all)
피스to가온_수금 = df_pistos['지사수금수수료'].sum()
#피스->가온 이동사원의 수금(공동정산대상)
피스to가온_수금2= df_pistos[(df_pistos['계약일자']>='20140101') & (df_pistos['계약일자']<'20161201')]['지사수금수수료'].sum()
print('피스to가온_수금 =', '{:,.0f}'.format( 피스to가온_수금))
print('피스to가온_수금2=', '{:,.0f}'.format( 피스to가온_수금2))

피스토스_수금정산금 = 피스토스_수금 + 피스to가온_수금 - ( 피스토스_수금2 * 0.3 + 피스to가온_수금2 * 0.3)
print(ym,'월','최종정산금= ', '{:,.0f}'.format(피스토스_수금정산금))
#정산 결과 파일 생성-------------------
file0 = f'{ym}_글래드피스토스정산금.xlsx'
dat = {'금액': [피스토스_수금,피스to가온_수금,피스토스_수금2,피스to가온_수금2],
     '글래드%': [100, 100, -30,  -30]}
index = ['피스토스수금','가온수금','피스토스(14~16.11)의30%','가온(14~16.11)의30%']
df_res = pd.DataFrame(dat, index)

df_res['정산지급액'] = df_res['금액'] * df_res['글래드%'] / 100
df_res.loc['Total',['금액','글래드%','정산지급액']] =df_res['금액'].sum(),0,df_res['정산지급액'].sum()

file0 = f'{ym}_글래드피스토스정산금.xlsx'
df_res.to_excel(dir0+file0)

# 미사용 -- openpxl
# import openpyxl
# wb = openpyxl.Workbook()
# ws = wb.active
# ws.cell(1,1).value = 'A. 수금수수료 합계'
# ws.cell(1,2).value = 피스토스_수금
# ws.cell(2,1).value = 'B. 수금수수료 합계(2014~2016.11)'
# ws.cell(2,2).value = 피스토스_수금2
# ws.cell(3,1).value = 'C. 가온 이동사원 수금 합계'
# ws.cell(3,2).value = 피스to가온_수금
# ws.cell(4,1).value = 'D. 가온 이동사원 수금 합계(2014~2016.11)'
# ws.cell(4,2).value = 피스to가온_수금2
# ws.cell(5,1).value = "E. 최종정산금= A + C - (B + D)*30%"
# ws.cell(5,2).value = 피스토스_수금정산금
# wb.save(dir0+file0)

print(">>> 피스토스 수금 정산금 파일 생성 완료")

피스토스_수금 = 4,724,347
피스토스_수금2= 1,063,105
피스to가온_수금 = 728,416
피스to가온_수금2= 273,591
202504 월 최종정산금=  5,051,754
>>> 피스토스 수금 정산금 파일 생성 완료


#### 피스토스 사원 수금명세 추출

In [4]:
brh = '글래드피스토스'  #지점
ins = '장기'           #보종
fc_name = ["고경숙","홍미숙"]

col = ['보험사','증권번호','계약종류','지점','사원번호','사원명','FC코드','계약일자','계약자명']
df_all = pd.DataFrame()
for idx, row in df[['보험사','수금 값열','수금값 제목']].iterrows():
    cols = col + [ row['수금값 제목'] ]
    if row['보험사'] !='계':
        print(row['보험사'], ':', row['수금값 제목'], " : processing") 
        df_temp = pd.DataFrame()
        df_temp = (pd.read_excel(ff,sheet_name=row['보험사'], usecols=cols, dtype={'증권번호': 'object','계약일자': 'object'})
                  .rename(columns={row['수금값 제목']:'총괄수금수수료'}))
        if brh:
            df_temp = df_temp[df_temp['지점']==brh]
        if ins:    
            df_temp = df_temp[df_temp['계약종류']==ins]
        df_all = pd.concat([df_all, df_temp])
df_all['수금수수료']= df_all['총괄수금수수료'] * .95

# 피스토스 수금은 95%값
df_wrt1 = df_all.drop(columns='총괄수금수수료')
# 사원수금은 100%값을 추출함 : 정산 오류가 있었음
# df_wrt2 = (df_all.query("계약일자 >= '20200801'  and  사원명 in (@fc_name) ")
#             .drop(columns='수금수수료')
#             .rename(columns={'총괄수금수수료':'수금수수료'}))
df_wrt2 = df_all.query("계약일자 >= '20200801'  and  사원명 in (@fc_name) ")
df_sum = pd.pivot_table(df_all, index='사원명',
               values=["총괄수금수수료","수금수수료"],
               aggfunc=sum,fill_value=0,margins=True, margins_name="Total").reset_index()
df_sum2 = pd.pivot_table(df_wrt2, index='사원명',
               values=["총괄수금수수료","수금수수료"],
               aggfunc=sum,fill_value=0,margins=True, margins_name="Total").reset_index()

file0 = f'{ym}_수금수수료명세(피스토스).xlsx'
with pd.ExcelWriter(dir0+file0, mode="w", engine='openpyxl') as writer:  
    df_sum.to_excel(writer, sheet_name='요약',index=False)
    df_wrt1.to_excel(writer, sheet_name='피스토스',index=False)
    df_sum2.to_excel(writer, sheet_name='요약(고경숙,홍미숙)',index=False)
    df_wrt2.to_excel(writer, sheet_name='사원(고경숙,홍미숙)',index=False)
    
print('end of job')

AIG손보 : 수금  : processing
DB손보 : 수금관리  : processing
MG손보 : 수금  : processing
흥국화재 : 장기고객관리  : processing
현대해상 : 수금  : processing
한화손보 : 장기고객관리  : processing
KB손보 : 수금  : processing
롯데손보 : 수금  : processing
메리츠화재 : 장기수금  : processing
농협손보 : 수금  : processing
삼성화재 : 장기수금  : processing
하나손보 : 수금수수료  : processing
end of job


In [ ]:
# pd.pivot_table?
df_sum = pd.pivot_table(df_all, index='지점',columns=['사원명'],
               values=["총괄수금수수료","수금수수료"],
               aggfunc=sum,fill_value=0,margins=True, margins_name="Total").reset_index()
df_sum.swaplevel(axis=1).sort_index(axis=1,ascending=[True,False])

In [22]:
df_sum2 = pd.pivot_table(df_wrt2, index='사원명',
               values=["총괄수금수수료","수금수수료"],
               aggfunc=sum,fill_value=0,margins=True, margins_name="Total").reset_index()
df_sum2

,사원명,수금수수료,총괄수금수수료
0,고경숙,152226.1,160238
1,홍미숙,166084.7,174826
2,Total,318310.8,335064
